# Supporting Vector Regression with variance based feature selection

In [9]:
# import necessary packages

# Plot the figures inline, necessary only for Jupyter notebook
%matplotlib inline 

import os # miscelleaneous operating system interface
import numpy as np  # import numpy
import pandas as pd # import pandas
import matplotlib.pyplot as plt 
import seaborn as sns # import seaborn for data visualization

from sklearn.metrics import root_mean_squared_error as RMSE # calculate RMSE
from sklearn.model_selection import train_test_split # splitting data into training and testing set

In [10]:
# import the data set for drugs

df_drug=pd.read_csv('GDSC2_label_14drugs.csv') # load the data set for drugs (limited to 14 drugs)
df_drug.set_index('Unnamed: 0', inplace=True)

# print(df_drug.shape) # shape for drug data

# Result: 805 tumor cells (cell lines) and 14 drugs
# df_drug.head(5) # print the first 5 instances to have a look

# We only focus on the 3 drugs with largest variances in their efficacies among different drugs
drug_sort=df_drug.std().sort_values(ascending = False).iloc[0:3]

# import the data set for tumor cells (cell lines) and genes

df_tumor=pd.read_csv('GDSC2_expression14.csv') # load the data set for tumors and cell lines
df_tumor.set_index('Unnamed: 0', inplace=True)
gene=list(df_tumor.columns)

# print('Number of genes:', len(gene))
# print('First gene: ',gene[0])

# print('Shape of data frame', df_tumor.shape) # shape for tumor data
# df_tumor.iloc[0:5, 0:10] # print the first 5 instances to have a look, only print the first 10 columns

# Result: 805 tumor cells (instances) and 17419 genes (features)

In [11]:
# The 3 drugs with the largest variances among all 805 tumors
# See EDA and overfitting notebook

drug1=drug_sort.index[0] # 'Docetaxel'
drug2=drug_sort.index[1] # 'Trametinib'
drug3=drug_sort.index[2] # 'Entinostat'

# Merge the two data set together WITHOUT any selection of features
# We do not need to save too many decimal places, keep 2 decimal places is fine
# The last column becomes the drug efficacy

df_1=pd.concat([df_tumor, df_drug[drug1].round(2)], axis=1) # axis=1 because we join the columns, not rows
df_2=pd.concat([df_tumor, df_drug[drug2].round(2)], axis=1) 
df_3=pd.concat([df_tumor, df_drug[drug3].round(2)], axis=1) 

# print(df_1.shape)
# df_1.head(5)

In [12]:
# Calculate variance for each gene across the 805 samples
# Select the 50 genes with highest variance (after normalization)

from sklearn.preprocessing import normalize # normalize the columns for the genes

n=50 # number of genes to keep

df_tumor_norm=pd.DataFrame(normalize(df_tumor, axis=0)) # result after normalization is a numpy array, we need data frame
df_tumor_norm.columns=gene # assign the gene name as column names

# calculate the variance for each gene type across 805 samples and sort the results
df_tumor_var=pd.DataFrame(df_tumor_norm.var())
df_tumor_var.columns=['normed var']

# Comments: There are many genes having small variances across different types of tumor cells. 
# Again, doesn't mean that they have no importance in drug efficacy!

# Picking the 50 genes with the largest variances across all tumor types

df_var=df_tumor_var.sort_values('normed var',ascending = False).iloc[0:n,:]
df_var.index # This list stores the names of that 50 genes
# df_var

Index(['RPS4Y1', 'HLA-DRA', 'ITM2A', 'MIR205HG', 'TACSTD2', 'SPP1', 'TSPAN8',
       'LAPTM5', 'TFF1', 'GMFG', 'COL1A2', 'KRT6A', 'LUM', 'S100A9', 'BEX1',
       'SRGN', 'CD53', 'IGJ', 'POU2AF1', 'S100A14', 'TFF3', 'S100P', 'GTSF1',
       'AKR1B10P1', 'COL3A1', 'C8orf4', 'UCA1', 'INSM1', 'GPX2', 'LCN2',
       'CEACAM5', 'MMP7', 'PCP4', 'CD52', 'LYZ', 'AKR1C2', 'FGFBP1', 'GRP',
       'PHGR1', 'MMP1', 'SCG2', 'RAB25', 'CXCR4', 'COL6A3', 'KRT5', 'POSTN',
       'BCL2A1', 'INHBB', 'CCL2', 'ANXA10'],
      dtype='object')

# SVR with hyperparameter tuning

1. For Docetaxel

In [13]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

df_train_1, df_test_1 = train_test_split(df_1, shuffle=True, random_state=42, test_size=.2) # For Docetaxel

# Hyperparameters
# C: L2 Regularization parameter ~ 1/C: Reduce if overfitting
# epsilon: width of the tube
# gamma: can be included in poly, rbf
# skip linear regression here

params = {'C': [0.05, 0.1, 0.15],  
              'gamma': [0.1, 0.01, 0.001,0.0001,0.00001], 
              'epsilon': [10, 1,0.1,0.01,0.001],
              'kernel': ['rbf']}  

# cv : number of k-fold cross validation

grid_search = GridSearchCV(SVR(),
                           param_grid=params,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           verbose=1)

grid_search.fit(df_train_1[df_var.index], df_train_1.iloc[:,-1:].values.ravel())
print(f'Best parameters found: {grid_search.best_params_}')

best_params = grid_search.best_params_
model = SVR(**best_params)
model.fit(df_train_1[df_var.index], df_train_1.iloc[:,-1:].values.ravel())

# Prediction on training data
y_train_pred = model.predict(df_train_1[df_var.index])
train_rmse = np.round(RMSE(df_train_1.iloc[:,-1:], y_train_pred),3)

# Prediction on test data
y_test_pred = model.predict(df_test_1[df_var.index])
test_rmse = np.round(RMSE(df_test_1.iloc[:,-1:], y_test_pred),3)

print(f'Training RMSE for {drug1}: {train_rmse}')
print(f'Test RMSE for {drug1}: {test_rmse}')

Fitting 3 folds for each of 75 candidates, totalling 225 fits
Best parameters found: {'C': 0.05, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
Training RMSE for Docetaxel: 0.172
Test RMSE for Docetaxel: 0.168


2. For Trametinib

In [14]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

df_train_2, df_test_2 = train_test_split(df_2, shuffle=True, random_state=42, test_size=.2) # For Trametinib

# Hyperparameters
# C: L2 Regularization parameter ~ 1/C: Reduce if overfitting
# epsilon: width of the tube
# gamma: can be included in poly, rbf
# skip linear regression here

params = {'C': [0.1, 0.15, 0.2, 0.25,0.3],  
              'gamma': [0.1, 0.01, 0.001,0.0001,0.00001], 
              'epsilon': [1,0.1,0.01,0.001],
              'kernel': ['rbf']}  

# cv : number of k-fold cross validation

grid_search = GridSearchCV(SVR(),
                           param_grid=params,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           verbose=1)

grid_search.fit(df_train_2[df_var.index], df_train_2.iloc[:,-1:].values.ravel())
print(f'Best parameters found: {grid_search.best_params_}')

best_params = grid_search.best_params_
model = SVR(**best_params)
model.fit(df_train_2[df_var.index], df_train_2.iloc[:,-1:].values.ravel())

# Prediction on training data
y_train_pred = model.predict(df_train_2[df_var.index])
train_rmse = np.round(RMSE(df_train_2.iloc[:,-1:], y_train_pred),3)

# Prediction on test data
y_test_pred = model.predict(df_test_2[df_var.index])
test_rmse = np.round(RMSE(df_test_2.iloc[:,-1:], y_test_pred),3)

print(f'Training RMSE for {drug2}: {train_rmse}')
print(f'Test RMSE for {drug2}: {test_rmse}')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found: {'C': 0.3, 'epsilon': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
Training RMSE for Trametinib: 0.066
Test RMSE for Trametinib: 0.192


3. For Entinostat

In [15]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

df_train_3, df_test_3 = train_test_split(df_3, shuffle=True, random_state=42, test_size=.2) # For Trametinib

# Hyperparameters
# C: L2 Regularization parameter ~ 1/C: Reduce if overfitting
# epsilon: width of the tube
# gamma: can be included in poly, rbf
# skip linear regression here

params = {'C': [0.04, 0.05, 0.06, 0.07, 0.08],  
              'gamma': [0.1, 0.01, 0.001,0.0001,0.00001], 
              'epsilon': [1,0.1,0.01,0.001],
              'kernel': ['rbf']}  

# cv : number of k-fold cross validation

grid_search = GridSearchCV(SVR(),
                           param_grid=params,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           verbose=1)

grid_search.fit(df_train_3[df_var.index], df_train_3.iloc[:,-1:].values.ravel())
print(f'Best parameters found: {grid_search.best_params_}')

best_params = grid_search.best_params_
model = SVR(**best_params)
model.fit(df_train_3[df_var.index], df_train_3.iloc[:,-1:].values.ravel())

# Prediction on training data
y_train_pred = model.predict(df_train_3[df_var.index])
train_rmse = np.round(RMSE(df_train_3.iloc[:,-1:], y_train_pred),3)

# Prediction on test data
y_test_pred = model.predict(df_test_3[df_var.index])
test_rmse = np.round(RMSE(df_test_3.iloc[:,-1:], y_test_pred),3)

print(f'Training RMSE for {drug3}: {train_rmse}')
print(f'Test RMSE for {drug3}: {test_rmse}')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found: {'C': 0.06, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
Training RMSE for Entinostat: 0.087
Test RMSE for Entinostat: 0.106
